In [ ]:
# ==============================================================================
# БЛОК 1: ИМПОРТ БИБЛИОТЕК И ОБЪЯВЛЕНИЕ ФУНКЦИЙ
# ==============================================================================

import pandas as pd
import re
from IPython.display import display

def get_industry_for_arxiv(primary_subject, title, abstract):
    """
    Классифицирует статью из arXiv по отрасли на основе основной категории (subject),
    а также ключевых слов в заголовке и аннотации.
    """
    # Объединяем текстовые поля для поиска по ключевым словам
    full_text = (str(title) + ' ' + str(abstract)).lower()

    # Сначала проверяем по основной категории (Primary_Subject) — это самый надежный способ
    if primary_subject:
        subject = primary_subject.lower()
        if subject.startswith('econ') or subject.startswith('q-fin'):
            return 'Экономика'
        if subject.startswith('q-bio'):
            return 'Биоинформатика'
        # Все категории Computer Science, Statistics (ML), и Electrical Engineering относим к Информатике
        if subject.startswith(('cs.', 'stat.ml', 'eess.')):
            return 'Информатика'

    # Если категория не дала однозначного ответа, ищем по ключевым словам
    econ_keywords = [
        'economic', 'finance', 'business', 'management', 'market',
        'portfolio', 'gdp', 'monetary', 'trade'
    ]
    bio_keywords = [
        'medical', 'health', 'disease', 'biodiversity', 'clinical',
        'neuro', 'dementia', 'aneurysm', 'biology', 'bioauto'
    ]

    if any(keyword in full_text for keyword in econ_keywords):
        return 'Экономика'
    if any(keyword in full_text for keyword in bio_keywords):
        return 'Биоинформатика'

    # Если ничего не подошло, но категория была из CS, возвращаем Информатику
    if primary_subject and primary_subject.lower().startswith('cs.'):
        return 'Информатика'

    return 'Прочее'

In [ ]:
# ==============================================================================
# БЛОК 2: ОСНОВНАЯ ЛОГИКА СКРИПТА
# ==============================================================================

file_path = 'arxiv.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        text_data = file.read()
    print(f"✅ Файл '{file_path}' успешно прочитан.")
except FileNotFoundError:
    print(f"❌ Ошибка: Файл '{file_path}' не найден!")
    print("➡️ Пожалуйста, загрузите файл в рабочую директорию и запустите ячейку снова.")
    text_data = None

if text_data:
    # Разделяем весь текст на отдельные записи по уникальному идентификатору "arXiv:"
    entries = re.split(r'(?=arXiv:)', text_data.strip())
    parsed_data = []
    print(f"Найдено {len(entries)} записей для обработки.")

    for entry in entries:
        if not entry.strip():
            continue

        try:
            # --- Извлечение данных с помощью регулярных выражений ---

            # ID статьи (e.g., 2510.07297)
            arxiv_id_match = re.search(r'arXiv:(\S+)', entry)
            arxiv_id = arxiv_id_match.group(1) if arxiv_id_match else None

            # Основная категория (e.g., cs.AI)
            subject_match = re.search(r'\[.+?\]\s+([\w\.-]+)', entry)
            primary_subject = subject_match.group(1) if subject_match else None

            # Год публикации
            year_match = re.search(r'Submitted \d+ \w+, (\d{4})', entry)
            year = int(year_match.group(1)) if year_match else None

            # Авторы
            authors_match = re.search(r'Authors:\s*(.+?)\nAbstract:', entry, re.DOTALL)
            authors = " ".join(authors_match.group(1).split()) if authors_match else "N/A"

            # Заголовок (находится между строкой с ID и строкой "Authors:")
            title_match = re.search(r'cs\.\w+\n(.*?)\nAuthors:', entry, re.DOTALL) # Общий случай
            if not title_match: # Если не нашлось, пробуем другой паттерн
                 title_match = re.search(r'\n(.*?)\nAuthors:', entry, re.DOTALL)
            title = " ".join(title_match.group(1).split()).strip() if title_match else "N/A"

            # Аннотация (для классификации)
            abstract_match = re.search(r'Abstract:\s*(.*?)\s*▽ More', entry, re.DOTALL)
            abstract = abstract_match.group(1).strip() if abstract_match else ""

            # Классификация
            industry = get_industry_for_arxiv(primary_subject, title, abstract)

            parsed_data.append({
                'ID': arxiv_id,
                'Title': title,
                'Authors': authors,
                'Year': year,
                'Primary_Subject': primary_subject,
                'Industry': industry,
                'Source': 'arXiv'
            })

        except Exception as e:
            print(f"❌ Ошибка при обработке записи, начинающейся с '{entry[:30]}...': {e}")

    if parsed_data:
        # Создаем DataFrame
        df = pd.DataFrame(parsed_data)

        # Убираем строки, где не удалось извлечь ключевую информацию
        df.dropna(subset=['ID', 'Title', 'Year'], inplace=True)
        df['Year'] = df['Year'].astype(int)

        # Сохраняем в файлы
        csv_filename = 'dataset2.csv'
        excel_filename = 'dataset2.xlsx'
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        df.to_excel(excel_filename, index=False)

        print(f"\n✅ Данные успешно обработаны и сохранены в файлы '{csv_filename}' и '{excel_filename}'.")
        print("\n📊 Первые 10 строк итоговой таблицы:")
        display(df.head(10))
    else:
        print("\n⚠️ В файле не найдено данных для обработки.")